# PROJETO - Big Data para Dados Públicos
## Professor Maciel Calebe
## Análise de Propagação de Doenças  


### Arthur Boschini da Fonseca, Caio de Camargo Aranha Tieri, Guilherme Farias Caproni
##### Descrição:

Este projeto tem como objetivo empregar os dados dos registros de atendimentos médicos a fim de monitorar e antecipar a disseminação de doenças infecciosas, como a gripe ou a dengue, ao longo do tempo e em diversas áreas geográficas.


##### Database Utilizada
*DATASUS - SRAG 2019*: Disponível em: https://opendatasus.saude.gov.br/dataset/srag-2019

##### Dicionário de Dados
Disponível em: https://d26692udehoye.cloudfront.net/SRAG/2019/dicionario_de_dados_SRAG_hospitalizado_2019.pdf




In [2]:
# Para utilizar recursos do sistema
import os
import sys

# Bibliotecas DASK
import dask
from dask.distributed import Client
import dask.dataframe as dd # similar ao pandas
import dask.multiprocessing

# Para plot
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

In [23]:
client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')
client

c:\Users\caiog\anaconda3\envs\big\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63442 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:63442/status,
Dashboard: http://127.0.0.1:63442/status,Workers: 2
Total threads: 2,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63445,Workers: 2
Dashboard: http://127.0.0.1:63442/status,Total threads: 2
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:63463,Total threads: 1
Dashboard: http://127.0.0.1:63464/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:63448,


In [24]:
df = dd.read_csv('INFLUD19.csv', encoding='latin-1', sep=';', dtype= {'DT_NOTIFIC': 'O', 'SEM_NOT': 'int64', 'DT_SIN_PRI': 'O', 'SEM_PRI': 'int64', 'SG_UF_NOT': 'O', 'ID_REGIONA': 'O', 'CO_REGIONA': 'float64', 'ID_MUNICIP': 'O', 'CO_MUN_NOT': 'int64', 'ID_UNIDADE': 'O', 'CO_UNI_NOT': 'int64', 'CS_SEXO': 'O', 'DT_NASC': 'O', 'NU_IDADE_N': 'int64', 'TP_IDADE': 'int64', 'COD_IDADE': 'int64', 'CS_GESTANT': 'int64', 'CS_RACA': 'float64', 'CS_ESCOL_N': 'float64', 'ID_PAIS': 'O', 'CO_PAIS': 'float64', 'SG_UF': 'O', 'CO_RG_RESI': 'float64', 'ID_MN_RESI': 'O', 'CO_MUN_RES': 'float64', 'CS_ZONA': 'float64', 'SURTO_SG': 'float64', 'NOSOCOMIAL': 'float64', 'AVE_SUINO': 'float64', 'FEBRE': 'float64', 'TOSSE': 'float64', 'GARGANTA': 'float64', 'DISPNEIA': 'float64', 'DESC_RESP': 'float64', 'SATURACAO': 'float64', 'DIARREIA': 'float64', 'VOMITO': 'float64', 'OUTRO_SIN': 'float64', 'OUTRO_DES': 'O', 'PUERPERA': 'float64', 'CARDIOPATI': 'float64', 'HEMATOLOGI': 'float64', 'SIND_DOWN': 'float64', 'HEPATICA': 'float64', 'ASMA': 'float64', 'DIABETES': 'float64', 'NEUROLOGIC': 'float64', 'PNEUMOPATI': 'float64', 'IMUNODEPRE': 'float64', 'RENAL': 'float64', 'OBESIDADE': 'float64', 'OBES_IMC': 'float64', 'OUT_MORBI': 'float64', 'MORB_DESC': 'O', 'VACINA': 'float64', 'DT_UT_DOSE': 'O', 'MAE_VAC': 'float64', 'DT_VAC_MAE': 'O', 'M_AMAMENTA': 'float64', 'DT_DOSEUNI': 'O', 'DT_1_DOSE': 'O', 'DT_2_DOSE': 'O', 'ANTIVIRAL': 'float64', 'TP_ANTIVIR': 'float64', 'OUT_ANTIV': 'O', 'DT_ANTIVIR': 'O', 'HOSPITAL': 'float64', 'DT_INTERNA': 'O', 'SG_UF_INTE': 'O', 'ID_RG_INTE': 'O', 'CO_RG_INTE': 'float64', 'ID_MN_INTE': 'O', 'CO_MU_INTE': 'float64', 'UTI': 'float64', 'DT_ENTUTI': 'O', 'DT_SAIDUTI': 'O', 'SUPORT_VEN': 'float64', 'RAIOX_RES': 'float64', 'RAIOX_OUT': 'O', 'DT_RAIOX': 'O', 'AMOSTRA': 'float64', 'DT_COLETA': 'O', 'TP_AMOSTRA': 'float64', 'OUT_AMOST': 'O', 'REQUI_GAL': 'O', 'IF_RESUL': 'float64', 'DT_IF': 'O', 'POS_IF_FLU': 'float64', 'TP_FLU_IF': 'float64', 'POS_IF_OUT': 'float64', 'IF_VSR': 'float64', 'IF_PARA1': 'float64', 'IF_PARA2': 'float64', 'IF_PARA3': 'float64', 'IF_ADENO': 'float64', 'IF_OUTRO': 'float64', 'DS_IF_OUT': 'O', 'LAB_IF': 'O', 'CO_LAB_IF': 'float64', 'PCR_RESUL': 'float64', 'DT_PCR': 'O', 'POS_PCRFLU': 'float64', 'TP_FLU_PCR': 'float64', 'PCR_FLUASU': 'float64', 'FLUASU_OUT': 'O', 'PCR_FLUBLI': 'float64', 'FLUBLI_OUT': 'O', 'POS_PCROUT': 'float64', 'PCR_VSR': 'float64', 'PCR_PARA1': 'float64', 'PCR_PARA2': 'float64', 'PCR_PARA3': 'float64', 'PCR_PARA4': 'float64', 'PCR_ADENO': 'float64', 'PCR_METAP': 'float64', 'PCR_BOCA': 'float64', 'PCR_RINO': 'float64', 'PCR_OUTRO': 'float64', 'DS_PCR_OUT': 'O', 'LAB_PCR': 'O', 'CO_LAB_PCR': 'float64', 'CLASSI_FIN': 'float64', 'CLASSI_OUT': 'O', 'CRITERIO': 'float64', 'EVOLUCAO': 'float64', 'DT_EVOLUCA': 'O', 'DT_ENCERRA': 'O', 'DT_DIGITA': 'O', 'HISTO_VGM': 'int64', 'PAIS_VGM': 'float64', 'CO_PS_VGM': 'float64', 'LO_PS_VGM': 'O', 'DT_VGM': 'float64', 'DT_RT_VGM': 'float64', 'PCR_SARS2': 'float64', 'PAC_COCBO': 'float64', 'PAC_DSCBO': 'float64'})

In [25]:
df.head()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DT_DIGITA,HISTO_VGM,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PCR_SARS2,PAC_COCBO,PAC_DSCBO
0,10/01/2019,2,06/01/2019,2,MG,BELO HORIZONTE,1449.0,BELO HORIZONTE,310620,UNIDADE DE PRONTO ATENDIMENTO LESTE,...,14/03/2019,0,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
1,03/01/2019,1,01/01/2019,1,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSP MUN M BOI MIRIM,...,<NA>,0,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
2,02/01/2019,1,31/12/2018,1,PE,1,1497.0,RECIFE,261160,IMIP,...,<NA>,0,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
3,10/01/2019,2,07/01/2019,2,SP,GVE XVII CAMPINAS,1342.0,CAMPINAS,350950,HOSPITAL MUNICIPAL DR MARIO GATTI CAMPINAS,...,<NA>,0,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
4,11/01/2019,2,06/01/2019,2,PE,4,1499.0,CARUARU,260410,SECRETARIA MUNICIPAL DE SAUDE DE CARUARU,...,<NA>,0,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN


Escolhendo colunas

In [31]:
informacoes_pessoais = df[["CS_SEXO", "DT_NASC", "CS_RACA", "SG_UF", "SURTO_SG", "NOSOCOMIAL", ]]
informacoes_pessoais = informacoes_pessoais.rename(columns={"CS_SEXO": "sexo", "DT_NASC": "data_nascimento", "CS_RACA": "raca", "SG_UF": "estado", "SURTO_SG": "surto_sindromegripal", "NOSOCOMIAL": "contraiu_no_hospital"})
informacoes_pessoais.head()

,sexo,data_nascimento,raca,estado,surto_sindromegripal,contraiu_no_hospital
0,M,17/03/1988,1.0,MG,2.0,2.0
1,F,30/05/2018,1.0,SP,9.0,NaN
2,M,07/05/2017,4.0,PE,9.0,9.0
3,F,15/07/2018,1.0,SP,2.0,2.0
4,F,15/09/2018,4.0,PE,9.0,9.0


Trocando as legendas do dataframe para valores normais

In [36]:
informacoes_pessoais['raca'] = informacoes_pessoais['raca'].replace({1.0: 'Branca', 2.0: 'Preta', 3.0: 'Amarela', 4.0: 'Parda', 5.0: 'Indígena', 9.0: 'Ignorado'})
informacoes_pessoais['surto_sindromegripal'] = informacoes_pessoais['surto_sindromegripal'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
informacoes_pessoais['contraiu_no_hospital'] = informacoes_pessoais['contraiu_no_hospital'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado', np.nan: 'Ignorado'})

In [37]:
informacoes_pessoais.head()

,sexo,data_nascimento,raca,estado,surto_sindromegripal,contraiu_no_hospital
0,M,17/03/1988,Branca,MG,Não,Não
1,F,30/05/2018,Branca,SP,Ignorado,Ignorado
2,M,07/05/2017,Parda,PE,Ignorado,Ignorado
3,F,15/07/2018,Branca,SP,Não,Não
4,F,15/09/2018,Parda,PE,Ignorado,Ignorado


Informações de contágio

In [47]:
sintomas = df[["FEBRE", "TOSSE", "GARGANTA", "DISPNEIA", "DESC_RESP", "SATURACAO", "DIARREIA", "VOMITO",  "OUTRO_SIN", "OUTRO_DES"]]

In [48]:
sintomas['FEBRE'] = sintomas['FEBRE'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['TOSSE'] = sintomas['TOSSE'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['GARGANTA'] = sintomas['GARGANTA'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['DISPNEIA'] = sintomas['DISPNEIA'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['DESC_RESP'] = sintomas['DESC_RESP'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['SATURACAO'] = sintomas['SATURACAO'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['DIARREIA'] = sintomas['DIARREIA'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['VOMITO'] = sintomas['VOMITO'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['OUTRO_SIN'] = sintomas['OUTRO_SIN'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['OUTRO_DES'] = sintomas['OUTRO_DES'].replace({np.nan: 'sem descrição'})

In [49]:
sintomas.head()

,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES
0,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Sim,DERRAME PLEURAL
1,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,sem descrição
2,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,sem descrição
3,Sim,Sim,Ignorado,Sim,Sim,Sim,Não,Não,Não,sem descrição
4,Sim,Não,Não,Sim,Sim,Sim,Ignorado,Ignorado,Não,sem descrição


Juntando as duas tabelas

In [51]:
#Juntando as duas tabelas: informacoes_pessoais e sintomas
df = dd.concat([informacoes_pessoais, sintomas], axis=1)
df.head()

c:\Users\caiog\anaconda3\envs\big\lib\site-packages\dask\dataframe\multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


,sexo,data_nascimento,raca,estado,surto_sindromegripal,contraiu_no_hospital,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES
0,M,17/03/1988,Branca,MG,Não,Não,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Sim,DERRAME PLEURAL
1,F,30/05/2018,Branca,SP,Ignorado,Ignorado,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,sem descrição
2,M,07/05/2017,Parda,PE,Ignorado,Ignorado,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,sem descrição
3,F,15/07/2018,Branca,SP,Não,Não,Sim,Sim,Ignorado,Sim,Sim,Sim,Não,Não,Não,sem descrição
4,F,15/09/2018,Parda,PE,Ignorado,Ignorado,Sim,Não,Não,Sim,Sim,Sim,Ignorado,Ignorado,Não,sem descrição
